# INST326 — Week 8 Exercises: Inheritance & Polymorphism (Library Management)

**Focus (Week 8 only):** subclassing, method overriding, `super()`, polymorphism via common method names, and composition vs. inheritance decisions in a small codebase.

**Out of scope (Week 9+):** abstract base classes, interfaces/protocols, multiple inheritance/mixins, advanced design patterns, decorators beyond basics, context managers beyond prior weeks, dependency injection, property descriptors beyond simple use.

> Context: Use the Library Management domain—books, members, loans, fines—to complete the tasks. Stick to basic single inheritance and straightforward overrides.


### Starter Scaffold (Week‑8‑safe)

Below is minimal starter code from prior weeks, extended slightly for Week 8. Feel free to modify it for the exercises. Avoid Week 9+ topics.


In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Dict, Optional, List

# --- Exceptions from Week 7 (basic) ---
class LibraryError(Exception): ...
class DuplicateBookError(LibraryError): ...
class OverdueLoanError(LibraryError): ...

# --- Base domain classes (no ABCs) ---
@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        """Default loan period for a generic book."""
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"

@dataclass
class Member:
    member_id: str
    email: str

    def max_concurrent_loans(self) -> int:
        return 5

    def describe(self) -> str:
        return f"Member<{self.member_id}>"

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False

    def mark_returned(self) -> None:
        self.returned = True

class Catalog:
    def __init__(self):
        self._books: Dict[str, Book] = {}

    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN already exists: {book.isbn}")
        if book.copies < 0:
            raise ValueError("copies must be non-negative")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

class LoanDesk:
    """Very small service; deliberately simple for Week 8 examples."""
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []

    def checkout(self, member: Member, book: Book) -> Loan:
        # naive stock check
        if book.copies <= 0:
            raise LibraryError("no available copies")
        book.copies -= 1
        due = datetime.now() + timedelta(days=book.loan_period_days())
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        return loan

    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned()


## 1) Subclass a Book type

Create a subclass `PrintedBook(Book)` that overrides `loan_period_days()` to 21 days and `describe()` to include the word 'Printed'.

In [ ]:
# Your code here
class PrintedBook(Book):
    def loan_period_days(self) -> int:
        return 21

    def describe(self) -> str:
        return f"Printed Book<{self.isbn}>: {self.title} (copies={self.copies})"

# Example:
# pb = PrintedBook("111", "Intro to Python", 2)
# pb.loan_period_days()  # 21
# pb.describe()          # includes 'Printed'

## 2) Another Book subtype

Create `EBook(Book)` that has an extra attribute `file_size_mb: float` (add to `__init__`), uses a 14‑day loan, and overrides `describe()` to show the size.

In [ ]:
# Your code here
@dataclass
class EBook(Book):
    file_size_mb: float = 0.0

    def loan_period_days(self) -> int:
        return 14

    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} (size={self.file_size_mb}MB, copies={self.copies})"

## 3) Override with super()

Create `AudioBook(Book)` with extra field `duration_min: int`. Override `describe()` to start with `super().describe()` and append `duration_min`.

In [ ]:
# Your code here
class AudioBook(Book):
    duration_min: int = 0

    def describe(self) -> str:
        base_desc = super().describe()
        return f"{base_desc}, duration={self.duration_min}min"
    ...

## 4) Non‑circulating subclass

Create `ReferenceBook(Book)` that **cannot** be checked out. Override `loan_period_days()` to return `0`. In `LoanDesk.checkout`, demonstrate a guard that raises `LibraryError('non-circulating')` if the period is 0.

In [ ]:
# Your code here
class ReferenceBook(Book):
    def loan_period_days(self) -> int:
        raise LibraryError("Reference books cannot be checked out")

# Update/extend LoanDesk.checkout minimally to guard non‑circulating

## 5) Member specialization

Create `Student(Member)` and `Staff(Member)`. Students can have 5 concurrent loans; staff 10. Override `max_concurrent_loans()` accordingly.

In [ ]:
# Your code here
class Student(Member):
    def max_concurrent_loans(self) -> int:
        return 3
    def describe(self) -> str:
        return f"Student<{self.member_id}>"
class Staff(Member):
    def max_concurrent_loans(self) -> int:
        """Staff can have up to 10 concurrent loans."""
        return 10

    def describe(self) -> str:
        return f"Staff<{self.member_id}>"
    ...

## 6) Polymorphic fine calculation

Write a function `late_fee(book: Book, days_late: int) -> float` that uses polymorphic behavior:
- PrintedBook: $0.25/day
- EBook: $0.10/day
- AudioBook: $0.15/day
- Fallback (Book): $0.20/day
Use `isinstance` checks only; do not modify the classes for this one.

In [ ]:
# Your code here
def late_fee(book: Book, days_late: int) -> float:
     if isinstance(book, PrintedBook):
        return 0.25 * days_late
     elif isinstance(book, EBook):
        return 0.10 * days_late
     elif isinstance(book, AudioBook):
        return 0.15 * days_late
     else:
        return 0.20 * days_late

## 7) Polymorphism without isinstance

Refactor your approach so **each subclass** implements `daily_late_fee()` and `late_fee(days_late)` (calling `daily_late_fee()`), then write a single function `compute_fee(book: Book, days_late: int)` that calls `book.late_fee(days_late)` without type checks.

In [ ]:
# Your code here
# Add methods to Book and subclasses; avoid breaking earlier exercises if possible.
@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        """Default loan period for a generic book."""
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"

    def daily_late_fee(self) -> float:
        """Default daily late fee for a generic book."""
        return 1.00

    def late_fee(self, days_late: int) -> float:
        """Calculate total late fee based on days overdue."""
        return self.daily_late_fee() * days_late


@dataclass
class PrintedBook(Book):
    def loan_period_days(self) -> int:
        """Printed books have a 21-day loan period."""
        return 21

    def describe(self) -> str:
        return f"Printed Book<{self.isbn}>: {self.title} (copies={self.copies})"

    def daily_late_fee(self) -> float:
        """Printed books cost $0.50/day when late."""
        return 0.50


@dataclass
class EBook(Book):
    file_size_mb: float = 0.0

    def loan_period_days(self) -> int:
        """EBooks have a 14-day loan period (same as base Book)."""
        return 14

    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} (size={self.file_size_mb}MB, copies={self.copies})"

    def daily_late_fee(self) -> float:
        """EBooks cost $0.25/day when late."""
        return 0.25


@dataclass
class AudioBook(Book):
    duration_min: int = 0

    def describe(self) -> str:
        base_desc = super().describe()
        return f"{base_desc}, duration={self.duration_min}min"

    def daily_late_fee(self) -> float:
        """AudioBooks cost $0.75/day when late."""
        return 0.75


@dataclass
class ReferenceBook(Book):
    def loan_period_days(self) -> int:
        """Reference books are non-circulating."""
        raise LibraryError("Reference books cannot be checked out")

    def describe(self) -> str:
        return f"Reference Book<{self.isbn}>: {self.title} (copies={self.copies})"


# Single function using polymorphism - no type checks!
def compute_fee(book: Book, days_late: int) -> float:
    """Compute late fee for any book type using polymorphism."""
    return book.late_fee(days_late)

## 8) Overriding __str__

Override `__str__` in `Book` to return `<Book isbn=... title=...>`. Override it in one subclass to include subtype info, e.g., `<PrintedBook isbn=...>`.

In [ ]:
# Your code here
# Override in Book and one subclass
@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 1.00

    def late_fee(self, days_late: int) -> float:
        return self.daily_late_fee() * days_late

    def __str__(self) -> str:
        return f"<Book isbn={self.isbn} title={self.title}>"


@dataclass
class PrintedBook(Book):
    def loan_period_days(self) -> int:
        return 21

    def describe(self) -> str:
        return f"Printed Book<{self.isbn}>: {self.title} (copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 0.50

    def __str__(self) -> str:
        return f"<PrintedBook isbn={self.isbn} title={self.title}>"

## 9) Composition vs. inheritance

Create a small `Notifier` class with method `notify(member: Member, message: str)`. Demonstrate **composition** by adding a `notifier` attribute to `LoanDesk` and using it during checkout to acknowledge a loan. Keep `Notifier` very simple (e.g., print or collect messages).

In [ ]:
# Your code here
class Notifier:
    def __init__(self):
        self.messages: List[str] = []

    def notify(self, member: Member, message: str) -> None:
        notification = f"To {member.email}: {message}"
        print(notification)
        self.messages.append(notification)

# Integrate into LoanDesk via composition (not inheritance)
class LoanDesk:
    def __init__(self, catalog: Catalog, notifier: Notifier):
        self.catalog = catalog
        self.loans: List[Loan] = []
        self.notifier = notifier

    def checkout(self, member: Member, book: Book) -> Loan:
        # naive stock check
        if book.copies <= 0:
            raise LibraryError("no available copies")
        book.copies -= 1
        due = datetime.now() + timedelta(days=book.loan_period_days())
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        self.notifier.notify(member, f"You have checked out '{book.title}'. It is due on {due.date()}.")
        return loan

    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned()
            member = Member(loan.member_id)

## 10) Enforcing limits polymorphically

Modify `LoanDesk.checkout` to check a member's current active loans (for that member_id) and compare to `member.max_concurrent_loans()` before allowing checkout. Demonstrate with a `Student` hitting the limit and a `Staff` not hitting it.

In [ ]:
# Your code here
class LoanDesk:
    def __init__(self, catalog: Catalog, notifier: Optional[Notifier] = None):
        self.catalog = catalog
        self.loans: List[Loan] = []
        self.notifier = notifier if notifier else Notifier()
    def checkout(self, member: Member, book: Book) -> Loan:
        active_loans = [loan for loan in self.loans 
                       if loan.member_id == member.member_id and not loan.returned]
        if len(active_loans) >= member.max_concurrent_loans():
            raise LibraryError(
                f"Member has reached maximum concurrent loans ({member.max_concurrent_loans()})"
            )
        
        loan_days = book.loan_period_days()
        
        if book.copies <= 0:
            raise LibraryError("no available copies")
        book.copies -= 1
        due = datetime.now() + timedelta(days=loan_days)
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        
        self.notifier.notify(
            member, 
            f"Checked out '{book.title}' (ISBN: {book.isbn}). Due: {due.strftime('%Y-%m-%d')}"
        )
        
        return loan
    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned()
# Update LoanDesk.checkout and show a brief demo

## 11) Subclass‑specific behavior

Add a method `download_link()` to `EBook` returning a fake URL string using the ISBN. Do not add this to `Book` or other subclasses. Show a short snippet where you use duck typing safely by checking `hasattr` before calling.

In [ ]:
# Your code here
@dataclass
class EBook(Book):
    file_size_mb: float = 0.0

    def loan_period_days(self) -> int:
        return 14

    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} (size={self.file_size_mb}MB, copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 0.25

    def download_link(self) -> str:
        """Generate a download link for the ebook."""
        return f"https://library.downloads.com/ebooks/{self.isbn}.epub"
# Add method to EBook and a usage demo with duck typing
def provide_access(book: Book) -> str:
    if hasattr(book, 'download_link'):
        return f"Access online: {book.download_link()}"
    else:
        return f"Pick up physical copy at circulation desk"

## 12) Polymorphic loan period by member type

Some libraries extend loan periods for `Staff`. Implement `effective_loan_period(book: Book, member: Member) -> int`:
- start from `book.loan_period_days()`
- if `isinstance(member, Staff)`, add +7 days
Return the resulting days.

In [ ]:
# Your code here
def effective_loan_period(book: Book, member: Member) -> int:
    base_period = book.loan_period_days()
    if isinstance(member, Staff):
        return base_period + 7
    else:
        return base_period

## 13) Override equality semantics (dataclass)

For `Book`, override `__eq__` so that books are considered equal iff ISBNs match (ignore title/copies). Write quick tests comparing a `PrintedBook` and `EBook` with the same ISBN—they should be equal by ISBN.

In [ ]:
# Your code here
@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 1.00

    def late_fee(self, days_late: int) -> float:
        return self.daily_late_fee() * days_late

    def __str__(self) -> str:
        return f"<Book isbn={self.isbn} title={self.title}>"

    def __eq__(self, other) -> bool:

        if not isinstance(other, Book):
            return False
        return self.isbn == other.isbn
# Override __eq__ on Book carefully; demonstrate with examples
print("=== Test 1: Different book types, same ISBN ===")
printed = PrintedBook(isbn="978-1234567890", title="Python Basics", copies=3)
ebook = EBook(isbn="978-1234567890", title="Python Advanced", copies=50, file_size_mb=5.2)
audio = AudioBook(isbn="978-1234567890", title="Python Audio", copies=10, duration_min=500)

print(f"PrintedBook == EBook: {printed == ebook}")  # True
print(f"EBook == AudioBook: {ebook == audio}")      # True
print(f"PrintedBook == AudioBook: {printed == audio}")  # True
print("All have same ISBN, so all are equal!\n")

print("=== Test 2: Same type, different ISBN ===")
book1 = PrintedBook(isbn="978-1111111111", title="Same Title", copies=5)
book2 = PrintedBook(isbn="978-2222222222", title="Same Title", copies=5)

print(f"book1 == book2: {book1 == book2}")  # False
print("Different ISBNs, so not equal (even with same title/copies)\n")

## 14) Draft a small class hierarchy diagram (markdown)

In **markdown**, sketch a tiny hierarchy diagram for `Book <- PrintedBook | EBook | AudioBook | ReferenceBook` and `Member <- Student | Staff`. No code—just a clear diagram using text/ASCII.

In [ ]:
# (Write your diagram in this markdown cell by editing it after running the notebook.)
Library Class Hierarchies
=========================

Book Hierarchy:
---------------
                    Book
                     |
        +------------+------------+------------+
        |            |            |            |
   PrintedBook    EBook      AudioBook   ReferenceBook
   
   - PrintedBook: 21-day loan, $0.50/day late fee
   - EBook: 14-day loan, $0.25/day late fee, has download_link()
   - AudioBook: 14-day loan, $0.75/day late fee, has duration
   - ReferenceBook: non-circulating (cannot checkout)


Member Hierarchy:
-----------------
                   Member
                     |
                +----+----+
                |         |
             Student    Staff
             
   - Student: max 5 concurrent loans
   - Staff: max 10 concurrent loans, +7 days bonus on all loans


Composition Relationship:
-------------------------
   LoanDesk
      |
      +-- has-a --> Catalog
      |
      +-- has-a --> Notifier
      |
      +-- has-a --> List[Loan]

## 15) Replace conditional with polymorphism

Currently, `LoanDesk.checkout` always uses `book.loan_period_days()`. Add an overridable method `checkout_message()` to `Book` and override it in at least two subclasses to customize the user‑facing message returned by `LoanDesk.checkout` (e.g., 'Enjoy your audiobook!'). Show the different messages without `if/elif` chains.

In [ ]:
# Your code here
@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 1.00

    def late_fee(self, days_late: int) -> float:
        return self.daily_late_fee() * days_late

    def __str__(self) -> str:
        return f"<Book isbn={self.isbn} title={self.title}>"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Book):
            return False
        return self.isbn == other.isbn

    def checkout_message(self) -> str:
        return "Enjoy your book!"

@dataclass
class PrintedBook(Book):
    def loan_period_days(self) -> int:
        return 21

    def describe(self) -> str:
        return f"Printed Book<{self.isbn}>: {self.title} (copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 0.50

    def __str__(self) -> str:
        return f"<PrintedBook isbn={self.isbn} title={self.title}>"

    def checkout_message(self) -> str:
        return "Remember to return your printed book on time!"
    
    @dataclass
    class EBook(Book):
     file_size_mb: float = 0.0

    def loan_period_days(self) -> int:
        return 14

    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} (size={self.file_size_mb}MB, copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 0.25

    def download_link(self) -> str:
        return f"https://library.downloads.com/ebooks/{self.isbn}.epub"

    def checkout_message(self) -> str:
        return f"Enjoy your ebook! Download link: {self.download_link()}"
@dataclass
class AudioBook(Book):
    duration_min: int = 0

    def describe(self) -> str:
        base_desc = super().describe()
        return f"{base_desc}, duration={self.duration_min}min"

    def daily_late_fee(self) -> float:
        return 0.75

    def checkout_message(self) -> str:
        return "Enjoy listening to your audiobook!"
    
    @dataclass
    class ReferenceBook(Book):
     def loan_period_days(self) -> int:
        raise LibraryError("Reference books cannot be checked out")

    def describe(self) -> str:
        return f"Reference Book<{self.isbn}>: {self.title} (copies={self.copies})"
    
    class LoanDesk:
     def __init__(self, catalog: Catalog, notifier: Optional[Notifier] = None):
        self.catalog = catalog
        self.loans: List[Loan] = []
        self.notifier = notifier if notifier else Notifier()

     def checkout(self, member: Member, book: Book) -> Loan:
       
        active_loans = [loan for loan in self.loans 
                       if loan.member_id == member.member_id and not loan.returned]
        if len(active_loans) >= member.max_concurrent_loans():
            raise LibraryError(
                f"Member has reached maximum concurrent loans ({member.max_concurrent_loans()})"
            )
        

        loan_days = book.loan_period_days()
        
 
        if book.copies <= 0:
            raise LibraryError("no available copies")
        book.copies -= 1
        due = datetime.now() + timedelta(days=loan_days)
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        
        
        message = book.checkout_message()
        
        self.notifier.notify(
            member, 
            f"Checked out '{book.title}'. Due: {due.strftime('%Y-%m-%d')}. {message}"
        )
        
        return loan

    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned()

## 16) Subclass‑specific stock policy

Override `LoanDesk.checkout` to deny checkout of `EBook` if `copies < 0` (simulate licensing depletion), but allow `PrintedBook` as long as `copies > 0`. Implement this by relying on each subclass's own `can_checkout(stock: int) -> bool` method. Default in `Book` should be `stock > 0`.

In [ ]:
# Your code here
# Add can_checkout to Book and subclasses; update LoanDesk.checkout accordingly
@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 1.00

    def late_fee(self, days_late: int) -> float:
        return self.daily_late_fee() * days_late

    def __str__(self) -> str:
        return f"<Book isbn={self.isbn} title={self.title}>"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Book):
            return False
        return self.isbn == other.isbn

    def checkout_message(self) -> str:
        return "Enjoy your book!"

    def can_checkout(self, stock: int) -> bool:
        return stock > 0



@dataclass
class PrintedBook(Book):
    def loan_period_days(self) -> int:
        return 21

    def describe(self) -> str:
        return f"Printed Book<{self.isbn}>: {self.title} (copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 0.50

    def __str__(self) -> str:
        return f"<PrintedBook isbn={self.isbn} title={self.title}>"

    def checkout_message(self) -> str:
        return "Enjoy your printed book! Please handle with care."



@dataclass
class EBook(Book):
    file_size_mb: float = 0.0

    def loan_period_days(self) -> int:
        return 14

    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} (size={self.file_size_mb}MB, copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 0.25

    def download_link(self) -> str:
        return f"https://library.downloads.com/ebooks/{self.isbn}.epub"

    def checkout_message(self) -> str:
        return f"Enjoy your ebook! Download link: {self.download_link()}"

    def can_checkout(self, stock: int) -> bool:
        return stock >= 0



@dataclass
class AudioBook(Book):
    duration_min: int = 0

    def describe(self) -> str:
        base_desc = super().describe()
        return f"{base_desc}, duration={self.duration_min}min"

    def daily_late_fee(self) -> float:
        return 0.75

    def checkout_message(self) -> str:
        return f"Enjoy your audiobook! Total duration: {self.duration_min} minutes."


@dataclass
class ReferenceBook(Book):
    def loan_period_days(self) -> int:
        raise LibraryError("Reference books cannot be checked out")

    def describe(self) -> str:
        return f"Reference Book<{self.isbn}>: {self.title} (copies={self.copies})"



class LoanDesk:
    def __init__(self, catalog: Catalog, notifier: Optional[Notifier] = None):
        self.catalog = catalog
        self.loans: List[Loan] = []
        self.notifier = notifier if notifier else Notifier()

    def checkout(self, member: Member, book: Book) -> Loan:

        active_loans = [loan for loan in self.loans 
                       if loan.member_id == member.member_id and not loan.returned]
        if len(active_loans) >= member.max_concurrent_loans():
            raise LibraryError(
                f"Member has reached maximum concurrent loans ({member.max_concurrent_loans()})"
            )
        
     
        loan_days = book.loan_period_days()
        
        
        if not book.can_checkout(book.copies):
            raise LibraryError("no available copies")
        
        book.copies -= 1
        due = datetime.now() + timedelta(days=loan_days)
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        
       
        message = book.checkout_message()
        

        self.notifier.notify(
            member, 
            f"Checked out '{book.title}'. Due: {due.strftime('%Y-%m-%d')}. {message}"
        )
        
        return loan

    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned()



print("=== Demonstrating can_checkout() polymorphism ===\n")

catalog = Catalog()
desk = LoanDesk(catalog)


printed = PrintedBook(isbn="978-1111", title="Physical Python", copies=1)
ebook = EBook(isbn="978-2222", title="Digital Python", copies=0, file_size_mb=2.5)

catalog.add_book(printed)
catalog.add_book(ebook)

student = Student(member_id="S001", email="student@example.com")


## 17) Sorting polymorphically

Create a list mixing `PrintedBook`, `EBook`, and `AudioBook`. Implement a function `sort_books_for_display(books: list[Book]) -> list[Book]` that sorts by this precedence: Printed first, then EBook, then AudioBook; ties broken by title. Use a key function that relies on `isinstance` or a small polymorphic `display_rank()` method.

In [ ]:

@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 1.00

    def late_fee(self, days_late: int) -> float:
        return self.daily_late_fee() * days_late

    def __str__(self) -> str:
        return f"<Book isbn={self.isbn} title={self.title}>"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Book):
            return False
        return self.isbn == other.isbn

    def checkout_message(self) -> str:
        return "Enjoy your book!"

    def can_checkout(self, stock: int) -> bool:
        return stock > 0

    def display_rank(self) -> int:
        return 99


@dataclass
class PrintedBook(Book):
    def loan_period_days(self) -> int:
        return 21

    def describe(self) -> str:
        return f"Printed Book<{self.isbn}>: {self.title} (copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 0.50

    def __str__(self) -> str:
        return f"<PrintedBook isbn={self.isbn} title={self.title}>"

    def checkout_message(self) -> str:
        return "Enjoy your printed book! Please handle with care."

    def display_rank(self) -> int:
        return 1


@dataclass
class EBook(Book):
    file_size_mb: float = 0.0

    def loan_period_days(self) -> int:
        return 14

    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} (size={self.file_size_mb}MB, copies={self.copies})"

    def daily_late_fee(self) -> float:
        return 0.25

    def download_link(self) -> str:
        return f"https://library.downloads.com/ebooks/{self.isbn}.epub"

    def checkout_message(self) -> str:
        return f"Enjoy your ebook! Download link: {self.download_link()}"

    def can_checkout(self, stock: int) -> bool:
        return stock >= 0

    def display_rank(self) -> int:
        return 2


@dataclass
class AudioBook(Book):
    duration_min: int = 0

    def describe(self) -> str:
        base_desc = super().describe()
        return f"{base_desc}, duration={self.duration_min}min"

    def daily_late_fee(self) -> float:
        return 0.75

    def checkout_message(self) -> str:
        return f"Enjoy your audiobook! Total duration: {self.duration_min} minutes."

    def display_rank(self) -> int:
        return 3


@dataclass
class ReferenceBook(Book):
    def loan_period_days(self) -> int:
        raise LibraryError("Reference books cannot be checked out")

    def describe(self) -> str:
        return f"Reference Book<{self.isbn}>: {self.title} (copies={self.copies})"



def sort_books_for_display(books: list[Book]) -> list[Book]:
    return sorted(books, key=lambda book: (book.display_rank(), book.title))



## 18) Minimal polymorphic report

Write `summarize_books(books: list[Book]) -> list[str]` that returns `describe()` for each. Show that the correct overridden `describe()` is used without `if/elif`.

In [ ]:
# Your code here
def summarize_books(books: list[Book]) -> list[str]:
     return [book.describe() for book in books]

## 19) Unit test: overriding works

Using `unittest`, add a small test class that checks `loan_period_days()` for `PrintedBook` (21) and `ReferenceBook` (0), and that `__str__` includes the subclass name for one subtype.

In [ ]:
# Your code here
import unittest
class PrintedBook(Book):
    def loan_period_days(self) -> int:
        return 21

    def __str__(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title}"

class ReferenceBook(Book):
    def loan_period_days(self) -> int:
        return 0

    def __str__(self) -> str:
        return f"ReferenceBook<{self.isbn}>: {self.title}"
class TestWeek8Inheritance(unittest.TestCase):
 def test_loan_periods_and_str(self):
        printed = PrintedBook("111", "Printed Book Example")
        reference = ReferenceBook("222", "Reference Book Example")

        self.assertEqual(printed.loan_period_days(), 21)
        self.assertEqual(reference.loan_period_days(), 0)

        self.assertIn("PrintedBook", str(printed))
        self.assertIn("ReferenceBook", str(reference))
# # To run inside notebook:
# # unittest.main(argv=['-v'], exit=False)

## 20) Polymorphic fee scenario (end‑to‑end)

Create a short demo that:
- Builds a `Catalog` and `LoanDesk` (with `Notifier` if you implemented it)
- Adds one book of each subtype
- Checks each out to a `Student`
- Simulates `days_late` values and prints fees using your polymorphic fee API
Show that different subtypes yield different fees without `if/elif` at the call site.

In [ ]:
# Your code here
from datetime import datetime, timedelta


class PrintedBook(Book):
    def loan_period_days(self) -> int:
        return 21

    def late_fee(self, days_late: int) -> float:
        return 0.25 * days_late  

    def __str__(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title}"

class ReferenceBook(Book):
    def loan_period_days(self) -> int:
        return 0  

    def late_fee(self, days_late: int) -> float:
        return 10.00 if days_late > 0 else 0.0  

    def __str__(self) -> str:
        return f"ReferenceBook<{self.isbn}>: {self.title}"

class Ebook(Book):
    def loan_period_days(self) -> int:
        return 14

    def late_fee(self, days_late: int) -> float:
        return 0.10 * days_late  

    def __str__(self) -> str:
        return f"Ebook<{self.isbn}>: {self.title}"

class Student(Member):
    pass 
# End-to-end demo using your polymorphic methods
def demo_polymorphic_fees():
    catalog = Catalog()
    desk = LoanDesk(catalog)
    student = Student("stu01", "student@umd.edu")

    printed = PrintedBook("111", "Printed Book Example", copies=1)
    reference = ReferenceBook("222", "Reference Manual", copies=1)
    ebook = Ebook("333", "Digital Learning Book", copies=1)

    for b in [printed, reference, ebook]:
        catalog.add_book(b)

    loans = [desk.checkout(student, b) for b in [printed, reference, ebook]]

    days_late_list = [3, 2, 5]
    print("---- Late Fee Summary ----")
    for book, days_late in zip([printed, reference, ebook], days_late_list):
        fee = book.late_fee(days_late)
        print(f"{book}: {days_late} days late → Fee: ${fee:.2f}")


## Python skills you'll need (Weeks 1–8)

- **Core syntax & data types:** variables, strings, numbers, booleans
- **Collections:** lists, dicts (basic use), simple list/dict comprehensions
- **Control flow:** `if/elif/else`, `for`, `while`
- **Functions & modules:** defining functions, parameters, returns, imports
- **File I/O & JSON (basic):** open/read/write, simple JSON usage
- **Classes & objects (Weeks 4–6):** defining classes, attributes, methods, `__init__`, `__str__`
- **Encapsulation basics:** simple validation; naming conventions for "private" attributes
- **Methods:** instance/class/static methods (as introduced up to Week 6)
- **Error handling & testing (Week 7):** `try/except/else/finally`, custom exceptions, basic `unittest`
- **Week 8 focus:** **single inheritance**, **method overriding**, **`super()`**, **polymorphism via common methods**, and **composition vs. inheritance** decisions
- **Standard library familiarity:** `datetime`, `timedelta`, built‑in exceptions
